# Setup

In [24]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
path['project'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification
path['images'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images
path['models'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models
path['expository'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/expository
path['literature'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/literature
path['notebooks'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/notebooks
path['presentation'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/presentation
path['scripts'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts


<a id='model_setup'></a>
# Model setup

In [25]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = False

val_expansion_factor: Union[None,int] = 3

sample_size: Union[None, dict] = {"mel": 2000,     # Handling class imbalance by upsampling minority classes/downsampling majority classes
                                  "bcc": 2000,     # Specify how many images of each lesion diagnosis we want in our training set.
                                  "akiec": 2000,
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.

In [26]:
# Create an instance of the process class with attribute values as above.
balance_ta = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,
                  val_expansion_factor=val_expansion_factor,
                  sample_size=sample_size,)

- Loaded file '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images/metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'bkl': 0, 'df': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Balancing classes in training set.
- Balanced training set (uses as many different images per lesion as possible): self.df_train
- Expanding validation set: will combine 3 predictions into one, for each lesion in val set.
- Expanded validation set (one image per lesion, repeated 3 times): self.df_val1
- Expanded validation set (use up to 3 different images per lesion, if available): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, se

In [27]:
import torchvision.transforms as transforms
transform = transforms.Compose([
#transforms.RandomCrop((300, 300)),
# transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [28]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = balance_ta        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "effnet"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "nocrop" # Something descriptive and unique for future reference. Default empty string "".

model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
#model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
#unfreeze_last: Union[None, bool] = True
# overwrite: Union[None, bool] = True

In [29]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

effnet_balance_ta = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,)

New files will be created. 
Base filename: effnet_ta_bal_10e_nocrop_00
Attributes saved to file: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/effnet_ta_bal_10e_nocrop_00_attributes.json


In [30]:
import time

tic = time.time()
effnet_balance_ta.train()
toc = time.time()
print(f"Elapsed time: {toc - tic}")

Unfrozen layer: features.6.0.block.0.0.weight
Unfrozen layer: features.6.0.block.0.1.weight
Unfrozen layer: features.6.0.block.0.1.bias
Unfrozen layer: features.6.0.block.1.0.weight
Unfrozen layer: features.6.0.block.1.1.weight
Unfrozen layer: features.6.0.block.1.1.bias
Unfrozen layer: features.6.0.block.2.fc1.weight
Unfrozen layer: features.6.0.block.2.fc1.bias
Unfrozen layer: features.6.0.block.2.fc2.weight
Unfrozen layer: features.6.0.block.2.fc2.bias
Unfrozen layer: features.6.0.block.3.0.weight
Unfrozen layer: features.6.0.block.3.1.weight
Unfrozen layer: features.6.0.block.3.1.bias
Unfrozen layer: features.6.1.block.0.0.weight
Unfrozen layer: features.6.1.block.0.1.weight
Unfrozen layer: features.6.1.block.0.1.bias
Unfrozen layer: features.6.1.block.1.0.weight
Unfrozen layer: features.6.1.block.1.1.weight
Unfrozen layer: features.6.1.block.1.1.bias
Unfrozen layer: features.6.1.block.2.fc1.weight
Unfrozen layer: features.6.1.block.2.fc1.bias
Unfrozen layer: features.6.1.block.2.f

In [31]:
# import torch
# import torch.nn as nn

# instance = rn18_balance_ta

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [32]:
# from utils import print_header
from multiclass_models import get_probabilities

instance = effnet_balance_ta

tic = time.time()

instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
                                                   data_dir=instance.data_dir,
                                                   model_dir=instance.model_dir,
                                                   model=instance.model,
                                                   filename=instance._filename,
                                                   label_codes=instance.label_codes,
                                                   transform=instance.transform,
                                                   batch_size=instance.batch_size,
                                                   Print=False,
                                                   save_as=instance._filename + "_val1",)

toc = time.time()

print(f"Elapsed time: {toc - tic}")

instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
                                                    data_dir=instance.data_dir,
                                                    model_dir=instance.model_dir,
                                                    model=instance.model,
                                                    filename=instance._filename,
                                                    label_codes=instance.label_codes,
                                                    transform=instance.transform,
                                                    batch_size=instance.batch_size,
                                                    Print=False,
                                                    save_as=instance._filename + "_val_a",)
tic = time.time()

print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/effnet_ta_bal_10e_nocrop_00_val1_probabilities.csv
Elapsed time: 85.89207935333252
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/effnet_ta_bal_10e_nocrop_00_val_a_probabilities.csv
Elapsed time: 94.3751266002655


In [33]:
# instance = rn18_balance_ta

# file_path1 = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val1_probabilities.csv")
# file_path_a = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val_a_probabilities.csv")

# instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
# instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [34]:
from utils import print_header

instance = effnet_balance_ta

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125
1,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125
3,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431
1,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125
3,HAM_0001466,ISIC_0031633,bkl,0.762092,0.237846,0.000006,0.000055,0.000002
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189


In [35]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = effnet_balance_ta

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
1,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
3,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431,0,0
1,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431,0,0
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,0
3,HAM_0001466,ISIC_0031633,bkl,0.762092,0.237846,0.000006,0.000055,0.000002,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0


In [36]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = effnet_balance_ta
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,135,4,6,54,26,225,0.6
akiec,6,24,5,19,3,57,0.421053
bcc,6,7,59,9,1,82,0.719512
mel,4,0,2,125,23,154,0.811688
nv,28,0,13,171,"1,139","1,351",0.843079
All,179,35,85,378,"1,192","1,869",_
precision,0.75419,0.685714,0.694118,0.330688,0.955537,_,0.657663



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,140,3,4,53,25,225,0.622222
akiec,6,26,7,15,3,57,0.45614
bcc,6,8,58,9,1,82,0.707317
mel,9,1,1,124,19,154,0.805195
nv,24,0,13,168,"1,146","1,351",0.848261
All,185,38,83,369,"1,194","1,869",_
precision,0.756757,0.684211,0.698795,0.336043,0.959799,_,0.667473


In [37]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = effnet_balance_ta

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.792937,0.679066,NaN,NaN,0.666274,0.652472,0.657663,0.608255,0.947295,0.941495,0.918254



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.799358,0.687827,NaN,NaN,0.672691,0.66162,0.667473,0.619696,0.943264,0.943006,0.912679


In [38]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = effnet_balance_ta

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
1,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,1
3,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431,0,0
1,HAM_0002730,ISIC_0026769,bkl,0.465640,0.188622,0.342188,0.002119,0.001431,0,0
2,HAM_0002730,ISIC_0025661,bkl,0.112020,0.357830,0.176107,0.305919,0.048125,1,0
3,HAM_0001466,ISIC_0031633,bkl,0.762092,0.237846,0.000006,0.000055,0.000002,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.791172,0.000076,0.001125,0.207438,0.000189,0,0


In [39]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = effnet_balance_ta
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,131,5,7,57,25,225,0.582222
akiec,5,24,6,19,3,57,0.421053
bcc,5,7,60,9,1,82,0.731707
mel,4,0,1,126,23,154,0.818182
nv,28,0,13,181,"1,129","1,351",0.835677
All,173,36,87,392,"1,181","1,869",_
precision,0.757225,0.666667,0.689655,0.321429,0.95597,_,0.654025



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,135,4,5,57,24,225,0.6
akiec,5,26,8,15,3,57,0.45614
bcc,5,7,59,10,1,82,0.719512
mel,9,1,1,124,19,154,0.805195
nv,24,0,12,179,"1,136","1,351",0.840859
All,178,38,85,385,"1,183","1,869",_
precision,0.758427,0.684211,0.694118,0.322078,0.96027,_,0.66217


In [40]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = effnet_balance_ta

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.786517,0.677768,NaN,NaN,0.660809,0.647561,0.654025,0.600586,0.947295,0.941495,0.918254



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.791867,0.684341,NaN,NaN,0.668239,0.65613,0.66217,0.609926,0.943264,0.943006,0.912679
